In [1]:
from common.inputreader import InputReader, PuzzleWrapper

puzzle = PuzzleWrapper(year=int("2022"), day=int("17"))

puzzle.header()

# Pyroclastic Flow

[Open Website](https://adventofcode.com/2022/day/17)

In [2]:
from common.matrix import Direction


# helper functions
class Shape:
    def __init__(self, shape: list):
        self.shape = shape
        self.width = len(shape[0])
        self.height = len(shape)
        self.points = []
        for y, row in enumerate(shape):
            for x, cell in enumerate(row):
                if cell == "#":
                    self.points.append([x, y])

    def __str__(self):
        return "\n".join(self.shape)

    def __iter__(self):
        return iter(self.points)

    def copy(self):
        return Shape(self.shape)

    def collides(self, shapes: list) -> bool:
        for shape in shapes:
            for point in self.points:
                if point in shape.points:
                    return True
        return False

    def set_floor(self, floor: int):
        # move up to floor
        for point in self.points:
            point[1] += floor

    def move(self, direction: Direction, shapes: list, max_x: int) -> bool:
        # revert function
        def revert():
            for point in self.points:
                if direction == Direction.DOWN:
                    point[1] += 1
                elif direction == Direction.UP:
                    point[1] -= 1
                elif direction == Direction.LEFT:
                    point[0] += 1
                elif direction == Direction.RIGHT:
                    point[0] -= 1

        # move
        for point in self.points:
            if direction == Direction.DOWN:
                point[1] -= 1
            elif direction == Direction.UP:
                point[1] += 1
            elif direction == Direction.LEFT:
                point[0] -= 1
            elif direction == Direction.RIGHT:
                point[0] += 1

        # check if out of bounds
        for point in self.points:
            if point[0] < 0 or point[0] >= max_x:
                revert()
                return False

        # check if y is out of bounds
        if direction == Direction.DOWN:
            for point in self.points:
                if point[1] < 0:
                    revert()
                    return False

        # if collides, revert
        if self.collides(shapes):
            revert()
            return False

        return True


def domain_from_input(input: InputReader) -> (list, list):
    shapes = [[
        list("####")
    ], [
        list(".#."),
        list("###"),
        list(".#."),
    ], [
        list("###"),
        list("..#"),
        list("..#"),
    ], [
        list("#"),
        list("#"),
        list("#"),
        list("#")
    ], [
        list("##"),
        list("##")
    ]]
    directions = list(input.as_str())
    return directions, shapes


test_input, test_shapes = domain_from_input(puzzle.example(0))
print(test_input)
for shape in test_shapes:
    print(shape)

['>', '>', '>', '<', '<', '>', '<', '>', '>', '<', '<', '<', '>', '>', '<', '>', '>', '>', '<', '<', '<', '>', '>', '>', '<', '<', '<', '>', '<', '<', '<', '>', '>', '<', '>', '>', '<', '<', '>', '>']
[['#', '#', '#', '#']]
[['.', '#', '.'], ['#', '#', '#'], ['.', '#', '.']]
[['#', '#', '#'], ['.', '.', '#'], ['.', '.', '#']]
[['#'], ['#'], ['#'], ['#']]
[['#', '#'], ['#', '#']]


In [25]:
from functools import cache


# test case (part 1)
def part_1(reader: InputReader, max_shapes: int, debug: bool) -> (int, list):
    directions, shapes = domain_from_input(reader)
    if debug:
        print(directions)

    max_x = 7

    def next_shape():
        shape = shapes[len(history) % len(shapes)]
        shape = Shape(shape)
        max_y = find_max_y()
        shape.set_floor(max_y + 3)
        shape.move(Direction.RIGHT, [], max_x)
        shape.move(Direction.RIGHT, [], max_x)
        return shape

    def find_max_y():
        max_y = 0
        for shape in history:
            for point in shape:
                max_y = max(max_y, point[1] + 1)
        return max_y

    def print_shapes():
        max_y = find_max_y()
        for point in current_shape:
            max_y = max(max_y, point[1])

        for y in reversed(range(max_y + 2)):
            row = []
            for x in range(max_x):
                row.append(".")
            for shape in history:
                for point in shape:
                    if point[1] == y:
                        row[point[0]] = "#"

            for point in current_shape:
                if point[1] == y:
                    row[point[0]] = "@"

            print("".join(row))
        print()

    history = []
    height_history = [0]
    current_shape = next_shape()
    direction_counter = 0

    while len(history) < max_shapes:
        next_direction = directions[direction_counter % len(directions)]
        direction_counter += 1

        if debug:
            print(f"Direction: {next_direction}")
            print_shapes()

        # Move direction
        if next_direction == ">":
            current_shape.move(Direction.RIGHT, history, max_x)
        elif next_direction == "<":
            current_shape.move(Direction.LEFT, history, max_x)

        if debug:
            print(f"Direction: v")
            print_shapes()

        # Move down
        ok = current_shape.move(Direction.DOWN, history, max_x)

        if not ok:
            history.append(current_shape)
            height_history.append(find_max_y())
            current_shape = next_shape()

    if debug:
        print("Final")
        print_shapes()

    height_changes = [height_history[i] - height_history[i - 1] for i in range(1, len(height_history))]
    return find_max_y(), height_changes


result, _ = part_1(puzzle.example(0), 2022, False)
print(result)
assert result == 3068

3068


In [26]:
# real case (part 1)
result, _ = part_1(puzzle.input(), 2022, False)
print(result)
assert result == 3083

3083


In [31]:
# test case (part 2)
def find_repeating_pattern(height_changes: list) -> (list, int):
    n = len(height_changes)
    for length in range(2, n // 2 + 1):  # Start from length 2
        for start in range(n - 2 * length + 1):
            pattern = height_changes[start:start + length]
            repetitions = 0
            for i in range(start, n - length + 1, length):
                if height_changes[i:i + length] == pattern:
                    repetitions += 1
                else:
                    break
            if repetitions > 5:  # Ensure the pattern repeats
                return pattern, start
    return None, -1


def part_2(reader: InputReader, test_cycles: int, max_shapes: int, debug: bool) -> int:
    _, height_changes = part_1(reader, test_cycles, debug)
    pattern, index = find_repeating_pattern(height_changes)
    print(f"Pattern: {pattern}, Starting Index: {index}")

    if pattern is None:
        raise ValueError("No repeating pattern found")

    pattern_length = len(pattern)
    pattern_height = sum(pattern)

    # Calculate the number of full patterns and the remaining rocks
    full_patterns = (max_shapes - index) // pattern_length
    remaining_rocks = (max_shapes - index) % pattern_length

    # Calculate the total height
    initial_height = sum(height_changes[:index])
    total_height = initial_height + full_patterns * pattern_height + sum(pattern[:remaining_rocks])

    return total_height


result = part_2(puzzle.example(0), 1000, 1000000000000, False)
print(result)
assert result == 1514285714288

Pattern: [1, 3, 3, 4, 0, 1, 2, 3, 0, 1, 1, 3, 2, 2, 0, 0, 2, 3, 4, 0, 1, 2, 1, 2, 0, 1, 2, 1, 2, 0, 1, 3, 2, 0, 0], Starting Index: 15
1514285714288


In [ ]:
# real case (part 2)
result = part_2(puzzle.input(), 10000, 1000000000000, False)
print(result)

In [24]:
# print easters eggs
puzzle.print_easter_eggs()

## Easter Eggs

<span title="I am the man who arranges the blocks / that descend upon me from up above!">crushed</span> (I am the man who arranges the blocks / that descend upon me from up above!)